# Machine Learning

### Goal : Construction off an application where the user ask for a destination and the algorithm provide according to the detail of the place a destination in the word. The algorithm can use the different column and the user can filter the destination by country or category if he wants. 

### Goal: Pour une destination recommendée l'utilisateur pourra choisir parmis les restaurants proposés

Import the librairies

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import os
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import nltk
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("df_tourisme.csv")

In [3]:
df.head(5)

,Category,Place,Description,Pays,URL,Site,Hotel,Adresse,Detail,Note
0,Paysages paradisiaques,Tangalle au Sri Lanka,"Tangalle au Sri Lanka, ce sont de longues plag...",Sri Lanka,https://www.globe-trotting.com/top-des-paysage...,Globe-Trotting,Anantara Kalutara Resort,"St. Sebastian's Road, Katukurunda, 12000 Kalut...",In a secluded beachfront along Sri Lanka’s sou...,NaN
1,Paysages paradisiaques,"Gringo Perdido, Hôtel paradisiaque à El Remate...","Niché au cœur de la forêt tropicale du Petén, ...",Guatemala,https://www.globe-trotting.com/top-des-paysage...,Globe-Trotting,La Inmaculada Hotel,"14 Calle 7-88 Zona 10, Zona 10, 01009 Guatemal...","Situé dans le populaire quartier Zona 10, l'ét...",NaN
2,Paysages paradisiaques,Lagos au Portugal : Que faire ? Où loger ?,Lagos est une ville pittoresque et charmante d...,Portugal,https://www.globe-trotting.com/top-des-paysage...,Globe-Trotting,Living Graça,"Rua Particular à Rua Senhora do Monte nº 4, Sa...","Situé à Lisbonne, le Living Graça propose un h...",NaN
3,Paysages insolites du monde,La Pointe de la Jonction à Genève en Suisse,La Pointe de la Jonction est incontestablement...,Suisse,https://www.globe-trotting.com/photos-de-voyag...,Globe-Trotting,"Prize by Radisson, Affoltern Am Albis","OBSTGARTENSTRASSE 7, 8910 Affoltern, Suisse","Ouvert depuis octobre 2013, le Prize by Radiss...",7.9
4,Paysages insolites du monde,Le volcan Dallol dans le désert Danakil en Éth...,Le Volcan Dallol dans le désert du Danakil en ...,Éthiopie,https://www.globe-trotting.com/photos-de-voyag...,Globe-Trotting,አባት Guest House,"Cameroon Street, Bole, 1000 Addis-Abeba, Éthiopie","Featuring quiet street views, አባት Guest House ...",NaN


Preprocessing of the dataset

In [4]:
df.isna().sum()

Category         0
Place            0
Description      0
Pays             0
URL              0
Site             0
Hotel            0
Adresse          0
Detail           0
Note           110
dtype: int64

Translate reviews

In [5]:
from transformers import MarianMTModel, MarianTokenizer

# Charger le modèle et le tokenizer MarianMT
model_name = "Helsinki-NLP/opus-mt-fr-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Fonction pour traduire un texte
def translate_text(texts, tokenizer, model):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(**inputs)
    translations = [tokenizer.decode(t, skip_special_tokens=True) for t in translated_tokens]
    return translations

Translation evaluator

In [7]:
from sentence_transformers.evaluation import TranslationEvaluator

# Initialize the TranslationEvaluator using the same texts from two languages
translation_evaluator = TranslationEvaluator(
    source_sentences=df["Detail"],
    target_sentences=df["Detail_translated"],
    name="news-commentary-en-nl",
)
results = translation_evaluator(model)


print(translation_evaluator.primary_metric)

# => "news-commentary-en-nl_mean_accuracy"

print(results[translation_evaluator.primary_metric])

KeyError: 'Detail_translated'

In [7]:
#df["description_translated"] = df["Description"].apply(lambda x: translate_text([x], tokenizer, model)[0])


In [8]:
#df['Detail_translated'] = df["Detail"].apply(lambda x: translate_text([x], tokenizer, model)[0])

In [9]:
df.head(3)

,Category,Place,Description,Pays,URL,Site,Hotel,Adresse,Detail,Note
0,Paysages paradisiaques,Tangalle au Sri Lanka,"Tangalle au Sri Lanka, ce sont de longues plag...",Sri Lanka,https://www.globe-trotting.com/top-des-paysage...,Globe-Trotting,Anantara Kalutara Resort,"St. Sebastian's Road, Katukurunda, 12000 Kalut...",In a secluded beachfront along Sri Lanka’s sou...,NaN
1,Paysages paradisiaques,"Gringo Perdido, Hôtel paradisiaque à El Remate...","Niché au cœur de la forêt tropicale du Petén, ...",Guatemala,https://www.globe-trotting.com/top-des-paysage...,Globe-Trotting,La Inmaculada Hotel,"14 Calle 7-88 Zona 10, Zona 10, 01009 Guatemal...","Situé dans le populaire quartier Zona 10, l'ét...",NaN
2,Paysages paradisiaques,Lagos au Portugal : Que faire ? Où loger ?,Lagos est une ville pittoresque et charmante d...,Portugal,https://www.globe-trotting.com/top-des-paysage...,Globe-Trotting,Living Graça,"Rua Particular à Rua Senhora do Monte nº 4, Sa...","Situé à Lisbonne, le Living Graça propose un h...",NaN


In [10]:
#df.to_csv('df_tourisme_translated.csv', index=False, encoding='utf-8')


Preprocessing of the description of the place 

In [2]:
df = pd.read_csv("df_tourisme_translated.csv")

In [3]:
df['Combined_description'] = df['description_translated'].fillna('') + " " + df['Detail_translated'].fillna('')

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joali\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joali\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joali\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def preprocess_text(text):
    text = re.sub(r"[^\w\s]", "", text)
    text = text.lower()
    
    lemmatizer = WordNetLemmatizer()
    
    tokens = word_tokenize(text)
    
    stop_words = set(stopwords.words('english'))
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    # Retourner la liste de tokens
    return tokens

In [6]:
df['Processed_Description'] = df['Combined_description'].apply(preprocess_text)


In [7]:
df[['Processed_Description', "Combined_description"]].head(1)

,Processed_Description,Combined_description
0,"[tangalle, sri, lanka, long, deserted, beach, ...","Tangalle in Sri Lanka, they are long deserted ..."


In [8]:
df.head(1)

,Category,Place,Description,Pays,URL,Site,Hotel,Adresse,Detail,Note,Processed_Description,description_translated,Detail_translated,Combined_description
0,Paysages paradisiaques,Tangalle au Sri Lanka,"Tangalle au Sri Lanka, ce sont de longues plag...",Sri Lanka,https://www.globe-trotting.com/top-des-paysage...,Globe-Trotting,Anantara Kalutara Resort,"St. Sebastian's Road, Katukurunda, 12000 Kalut...",In a secluded beachfront along Sri Lanka’s sou...,NaN,"[tangalle, sri, lanka, long, deserted, beach, ...","Tangalle in Sri Lanka, they are long deserted ...","In a orcluded beachfront along Sri Lanka, only...","Tangalle in Sri Lanka, they are long deserted ..."


In [9]:
df['Processed_Description'] = df['Processed_Description'].apply(lambda x: " ".join(x))

In [10]:
from nltk.corpus import wordnet

def enrich_text_with_synonyms(text):
    words = text.split()
    synonyms = []
    for word in words:
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
    return " ".join(list(set(words + synonyms)))

df['Enriched_Text'] = df['Processed_Description'].apply(enrich_text_with_synonyms)


### Recommendations de destinations

Link to the model : https://huggingface.co/Prashasst/anime-recommendation-model

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Charger le modèle SentenceTransformer via Hugging Face
model = SentenceTransformer('Prashasst/anime-recommendation-model')


df['Processed_Description'] = df['Processed_Description'].astype(str)

# Génération des embeddings pour toutes les descriptions et la requête
embeddings = model.encode(df['Processed_Description'].tolist())

query = "Looking for a place where I can do a roadtrip."
query_embedding = model.encode([query])  

# Calculer les similarités cosinus
similarity_scores = cosine_similarity(query_embedding, embeddings)  # Donne (1, 171)

# Ajouter des scores au DataFrame
df['Similarity_Score'] = similarity_scores[0]  

# Afficher les résultats triés par Similarity_Score
result = df[['Place','Pays',"Adresse", "Hotel","description_translated", 'Similarity_Score']].sort_values(by='Similarity_Score', ascending=False)
result.head(5) 


c:\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\joali\.cache\huggingface\hub\models--sentence-transformers--bert-base-nli-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


,Place,Pays,Adresse,Hotel,description_translated,Similarity_Score
14,L'essentiel de l'Argentine en 15J / 14N,Argentine,"1227 Buenos Aires, 4630 Humahuaca, Argentine",Mestiza Hostal,This tour will guide you through the must-sees...,0.364078
19,Sapa,Vietnam,"259 Đường Võ Nguyên Giáp, Hô-Chi-Minh-Ville, V...",Green Lumiere Riverside Aparthotel Luxury,"In the province of Ciino Cai, on the border wi...",0.334233
153,Manger des Fourmis en Colombie,Colombie,"Calle de la Cruz No 9-111, San Diego, 130001 C...",Maloka Boutique Hostel,"During our 3-week trip to Colombia, we had the...",0.321590
155,Colombie : Itinéraire de 3 semaines,Colombie,"Calle de la Cruz No 9-111, San Diego, 130001 C...",Maloka Boutique Hostel,"Trip of 3 weeks to Colombia: Itinerary, stages...",0.316925
146,Est-ce dangereux de voyager en Jamaïque ?,Jamaïque,"176 Rhyne Park, Edmund Ridge Estate,, Montego ...",Sky View Villas @ Edmund Ridge Estate,Is it dangerous to travel to Jamaica? Do you f...,0.307666


### Tri des recommandations de destinations par pays 

In [18]:
# Tri par Similarity_Score de manière décroissante
result_sorted = df[['Pays', 'description_translated', 'Similarity_Score',"Place"]].sort_values(by='Similarity_Score', ascending=False)

# Afficher les destinations avec leurs scores de similarité
result_sorted[['Pays', 'description_translated', 'Similarity_Score']].head(5) # Afficher les 10 meilleures destinations


destinations_par_pays = result_sorted.groupby('Pays')[['Place','description_translated', 'Similarity_Score']].apply(lambda x: x.sort_values('Similarity_Score', ascending=False))
destinations_par_pays.head(5) # Afficher les meilleures destinations par pays

Place  \
Pays                                                     
Argentine 14   L'essentiel de l'Argentine en 15J / 14N   
Bolivie   5               Le Salar de Uyuni en Bolivie   
Colombie  164                        Guane en Colombie   
          163                  Santa Marta en Colombie   
          150          Le triangle du café en Colombie   

                                          description_translated  \
Pays                                                               
Argentine 14   This tour will guide you through the must-sees...   
Bolivie   5    The Salar d'Uyuni (or Salar de Uyuni in Spanis...   
Colombie  164  Guane in Colombia is a small peaceful village,...   
          163  On the Caribbean coast, the city of Santa Mart...   
          150  The Coffee Triangle in Colombia, also known as...   

               Similarity_Score  
Pays                             
Argentine 14           0.550829  
Bolivie   5            0.485286  
Colombie  164          0.538670  
          163          0.538530  
          150          0.538433

### Evaluation of the model 

Compute the MSEEvaluator

In [19]:
from sentence_transformers.evaluation import MSEEvaluator

sentences = query + df["Processed_Description"]
teacher_model = SentenceTransformer('all-mpnet-base-v2')

# Given queries, a corpus and a mapping with relevant documents, the InformationRetrievalEvaluator computes different IR metrics.
mse_evaluator = MSEEvaluator(
    source_sentences=sentences,
    target_sentences=sentences,
    teacher_model=teacher_model,
    name="stsb-dev",
)
results = mse_evaluator(model)



In [20]:
print(mse_evaluator.primary_metric)
# => "stsb-dev_negative_mse"
print(results[mse_evaluator.primary_metric])

stsb-dev_negative_mse
-0.12092443648725748


## Streamlit Application for the recommandation system

Countries csv

In [21]:
data = {
    "Country": ['Sri Lanka', 'Guatemala', 'Portugal', 'Suisse', 'Éthiopie', 'Bolivie', 'Namibie', 'France', 'Italie', 'Islande', 'Mexique', 'Argentine', 'Vietnam', 'Jamaïque', 'Colombie'],
    "Latitude": [7.8731, 15.7835, 39.3999, 46.8182, 9.145, -16.2902, -22.9576, 46.6034, 41.8719, 64.9631, 23.6345, -38.4161, 14.0583, 18.1096, 4.5709],
    "Longitude": [80.7718, -90.2308, -8.2245, 8.2275, 40.4897, -63.5887, 18.4904, 1.8883, 12.5674, -19.0208, -102.5528, -63.6167, 108.2772, -77.2975, -74.2973]
}

data = pd.DataFrame(data)

In [22]:
df = df.merge(data, left_on="Pays", right_on="Country", how="left")


In [24]:
df[df["Country"] == "Colombie"]

,Category,Place,Description,Pays,URL,Site,Hotel,Adresse,Detail,Note,Processed_Description,description_translated,Detail_translated,Combined_description,Enriched_Text,Similarity_Score,Country,Latitude,Longitude
31,Lieux mystiques,La Cathédrale de Sel de Zipaquirá en Colombie,"Située à 45 kilomètres de Bogotá en Colombie, ...",Colombie,https://www.globe-trotting.com/le-top-des-lieu...,Globe-Trotting,Maloka Boutique Hostel,"Calle de la Cruz No 9-111, San Diego, 130001 C...",Bénéficiant d'une connexion Wi-Fi gratuite et ...,8.3,located 45 kilometre bogotá colombia cathedral...,"Located 45 kilometres from Bogotá in Colombia,...","With free Wi-Fi and a terrace, Maloka Hostel C...","Located 45 kilometres from Bogotá in Colombia,...",military_service external site rivet fling fiv...,0.511367,Colombie,4.5709,-74.2973
149,Lieux mystiques du Monde,Que faire et que voir en Colombie ?,"La Colombie, joyau d'Amérique du Sud, est une ...",Colombie,https://www.globe-trotting.com/blog/categories...,Globe-Trotting,Maloka Boutique Hostel,"Calle de la Cruz No 9-111, San Diego, 130001 C...",Bénéficiant d'une connexion Wi-Fi gratuite et ...,8.3,colombia jewel south america land many facet e...,"Colombia, a jewel of South America, is a land ...","With free Wi-Fi and a terrace, Maloka Hostel C...","Colombia, a jewel of South America, is a land ...",military_service external fling five_hundred p...,0.489518,Colombie,4.5709,-74.2973
150,Lieux mystiques du Monde,Le triangle du café en Colombie,"Le Triangle du Café en Colombie, également con...",Colombie,https://www.globe-trotting.com/blog/categories...,Globe-Trotting,Maloka Boutique Hostel,"Calle de la Cruz No 9-111, San Diego, 130001 C...",Bénéficiant d'une connexion Wi-Fi gratuite et ...,8.3,coffee triangle colombia also known eje cafete...,"The Coffee Triangle in Colombia, also known as...","With free Wi-Fi and a terrace, Maloka Hostel C...","The Coffee Triangle in Colombia, also known as...",military_service trilateral external site flin...,0.538433,Colombie,4.5709,-74.2973
151,Lieux mystiques du Monde,Visite d'une Finca dans le triangle du café en...,"Lors de mon voyage en Colombie, j'ai eu l'occa...",Colombie,https://www.globe-trotting.com/blog/categories...,Globe-Trotting,Maloka Boutique Hostel,"Calle de la Cruz No 9-111, San Diego, 130001 C...",Bénéficiant d'une connexion Wi-Fi gratuite et ...,8.3,trip colombia opportunity spend day typical fi...,"During my trip to Colombia, I had the opportun...","With free Wi-Fi and a terrace, Maloka Hostel C...","During my trip to Colombia, I had the opportun...",military_service external raise fling five_hun...,0.515807,Colombie,4.5709,-74.2973
152,Lieux mystiques du Monde,Voyage en Colombie : Démarches et préparatifs,Vous êtes en train de planifier un périple en ...,Colombie,https://www.globe-trotting.com/blog/categories...,Globe-Trotting,Maloka Boutique Hostel,"Calle de la Cruz No 9-111, San Diego, 130001 C...",Bénéficiant d'une connexion Wi-Fi gratuite et ...,8.3,planning journey colombia country rich colour ...,"You are planning a journey in Colombia, a coun...","With free Wi-Fi and a terrace, Maloka Hostel C...","You are planning a journey in Colombia, a coun...",military_service external height fling product...,0.509993,Colombie,4.5709,-74.2973
153,Lieux mystiques du Monde,Manger des Fourmis en Colombie,Pendant notre voyage de 3 semaines en Colombie...,Colombie,https://www.globe-trotting.com/blog/categories...,Globe-Trotting,Maloka Boutique Hostel,"Calle de la Cruz No 9-111, San Diego, 130001 C...",Bénéficiant d'une connexion Wi-Fi gratuite et ...,8.3,3week trip colombia opportunity participate un...,"During our 3-week trip to Colombia, we had the...","With free Wi-Fi and a terrace, Maloka Hostel C...","During our 3-week trip to Colombia, we had the...",military_service external enter fling five_hun...,0.495544,Colombie,4.5709,-74.2973
154,Lieux mystiques du Monde,Le parc de Tayrona en Colombie,"Le Parc national naturel de Tayrona, bordé par...",Colombi